# PHME20 Data Challenge

## Table of Contents
* [Dataset Configuration Parameters](#dataset-configuration-parameters)
* [Global Parameters](#global-parameters)
* [Utility Functions](#utility-functions)
* [RUL Assignment](#rul-assignment)
* [Machine Learning Models](#ml-models)
    * [Cross Validation](#ml-cross-validation)
    * [Training](#ml-training)
    * [Evaluation](#ml-evaluation)



In [ ]:
# Import necessary Libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans

from ngboost import NGBRegressor
from ngboost.scores import MLE, CRPS
from sklearn.tree import DecisionTreeRegressor

from catboost import CatBoostRegressor

from sacred import Experiment
from sacred.commands import print_config
from sacred.observers import FileStorageObserver

from sklearn.cluster import KMeans

import pickle
import os

In [ ]:
ex = Experiment('PHME20', interactive=True)
ex.observers.append(FileStorageObserver('CB_kmeans/'))

# Dataset Configuration Parameters <a class="anchor" id="dataset-configuration-parameters"></a>

In [ ]:
input_path = "./input/"

train_small_path = input_path + "Training/Small/"
train_large_path = input_path + "Training/Large/"
validation_small_path = input_path + "Validation/Small/"
validation_large_path = input_path + "Validation/Large/"


In [ ]:
dataset_params = {
     1: {'path': train_small_path, 'size': 1, 'ratio': 0.4, 'profile': 1},
     2: {'path': train_small_path, 'size': 1, 'ratio': 0.4, 'profile': 1},
     3: {'path': train_small_path, 'size': 1, 'ratio': 0.4, 'profile': 1},
     4: {'path': train_small_path, 'size': 1, 'ratio': 0.4, 'profile': 1},
    
     5: {'path': train_small_path, 'size': 1, 'ratio': 0.425, 'profile': 2},
     6: {'path': train_small_path, 'size': 1, 'ratio': 0.425, 'profile': 2},
     7: {'path': train_small_path, 'size': 1, 'ratio': 0.425, 'profile': 2},
     8: {'path': train_small_path, 'size': 1, 'ratio': 0.425, 'profile': 2},

     9: {'path': train_small_path, 'size': 1, 'ratio': 0.45, 'profile': 3},
    10: {'path': train_small_path, 'size': 1, 'ratio': 0.45, 'profile': 3},
    11: {'path': train_small_path, 'size': 1, 'ratio': 0.45, 'profile': 3},
    12: {'path': train_small_path, 'size': 1, 'ratio': 0.45, 'profile': 3},
    
    13: {'path': validation_small_path, 'size': 1, 'ratio': 0.475, 'profile': 4},
    14: {'path': validation_small_path, 'size': 1, 'ratio': 0.475, 'profile': 4},
    15: {'path': validation_small_path, 'size': 1, 'ratio': 0.475, 'profile': 4},
    16: {'path': validation_small_path, 'size': 1, 'ratio': 0.475, 'profile': 4},
    
    33: {'path': train_large_path, 'size': 3, 'ratio': 0.4, 'profile': 9},
    34: {'path': train_large_path, 'size': 3, 'ratio': 0.4, 'profile': 9},
    35: {'path': train_large_path, 'size': 3, 'ratio': 0.4, 'profile': 9},
    36: {'path': train_large_path, 'size': 3, 'ratio': 0.4, 'profile': 9},
    
    37: {'path': train_large_path, 'size': 3, 'ratio': 0.425, 'profile': 10},
    38: {'path': train_large_path, 'size': 3, 'ratio': 0.425, 'profile': 10},
    39: {'path': train_large_path, 'size': 3, 'ratio': 0.425, 'profile': 10},
    40: {'path': train_large_path, 'size': 3, 'ratio': 0.425, 'profile': 10},
                                    
    41: {'path': train_large_path, 'size': 3, 'ratio': 0.45, 'profile': 11},
    42: {'path': train_large_path, 'size': 3, 'ratio': 0.45, 'profile': 11},
    43: {'path': train_large_path, 'size': 3, 'ratio': 0.45, 'profile': 11},
    44: {'path': train_large_path, 'size': 3, 'ratio': 0.45, 'profile': 11},

    45: {'path': validation_large_path, 'size': 3, 'ratio': 0.475, 'profile': 12},
    46: {'path': validation_large_path, 'size': 3, 'ratio': 0.475, 'profile': 12},
    47: {'path': validation_large_path, 'size': 3, 'ratio': 0.475, 'profile': 12},                                 
    48: {'path': validation_large_path, 'size': 3, 'ratio': 0.475, 'profile': 12},                                 

}


In [ ]:
experiments_for_training = {
    25 : ([1, 5, 9, 33, 37, 41],
          [13, 14, 15, 16, 45, 46, 47, 48]),
    50 : ([1, 2, 5, 6, 9, 10, 33, 34, 37, 38, 41, 42],
          [13, 14, 15, 16, 45, 46, 47, 48]),
    75 : ([1, 2, 3, 5, 6, 7, 9, 10, 11, 33, 34, 35, 37, 38, 39, 41, 42, 43],
          [13, 14, 15, 16, 45, 46, 47, 48]),
    100: ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44],
          [13, 14, 15, 16, 45, 46, 47, 48]),
}
  

In [ ]:
model_names = {
    25: 'model_4_25_percent.pkl',
    50: 'model_3_50_percent.pkl',
    75: 'model_2_75_percent.pkl',
    100: 'model_1_all_experiements.pkl',
}

# Global Parameters <a class="anchor" id="global-parameters"></a>

In [ ]:
@ex.config
def configuration_settings():
    RUL_Max = 150 # seconds!
    scale_type = "standard"
    RUL_Assignment_Method = "Linear"
    window_size = 5
    train_model = True
    eval_model = True
    data_percent = 1
    training_list, validating_list = experiments_for_training[data_percent]
    final_model_path = model_names[data_percent]
    

In [ ]:
# Globals
RUL_Assignment = None
GLOBAL_INIT_RUL = None # seconds!

# Main sensor list. 
# List shall include other features such as operating conditions. 
sensor_list = ['Flow_Rate', 'Upstream_Pressure', 'Downstream_Pressure', 'Pressure_Drop']
feature_list = ['Flow_Rate', 'Upstream_Pressure', 'Downstream_Pressure', 'Pressure_Drop', 'Particle_Size', 'Ratio', 'Kmeans_Profile']
experiment_params = ['Particle_Size', 'Ratio']
n_profiles = 6

verbose = False

# Utility Functions <a class="anchor" id="utility-functions"></a>

In [ ]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [ ]:
# Fault estimation from line detection
def fault_estimation(experiment_df, ws=3):
    flow_rate = running_mean(experiment_df.Flow_Rate.values, ws)
    count = flow_rate.shape[0]

    x_fr = list(range(flow_rate.shape[0]))

    # fit a polygon for readings b/w 400 and 1400
    z0 = np.polyfit(x_fr[400:1400], flow_rate[400:1400], 1) 
#     print (z0)

    line_func = np.poly1d(z0) # create function flow line params
    line = line_func(x_fr) # fit a line for polygon

    y_diff = line - flow_rate
    y_diff_rev = y_diff[::-1] # reverse the points of line

    # Find where the line intersects the Flow_Rate
    index = np.argmax(y_diff_rev < 0) 
    intersection = count - index 

    return int(intersection.round())

In [ ]:
# Create sequences based on window size
def create_sequence(sequence, window_size):
    X = list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + window_size
        # check if we are beyond the sequence
        if end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.array(X)
 

# Create dataset for experiment
def create_dataset_for_experiment(df, ws):
    data_data = np.empty((0, ws * len(feature_list)), dtype=np.float32) # for 1D
    rul_data = np.empty((0, 1), dtype=np.int)

    experiment_list = df.ExperimentID.unique()
    
    for experiment in experiment_list:
        experiment_df = df[df['ExperimentID'] == experiment]
        sensors_df = experiment_df.filter(feature_list)

        # Calculate seq of windows_size len
        seq = create_sequence(sensors_df.values, window_size=ws)
        seq_count = seq.shape[0]
        seq = seq.reshape((seq_count, -1)) # for 1D

        # add new seq to data_data array
        data_data = np.vstack((data_data, seq))

        # Calculate RULS
        rul_df = experiment_df[RUL_Assignment] * 1.0 #?
        ruls = rul_df.values[-seq_count:].reshape((seq_count, -1))

        # add rul to rul_data array
        rul_data = np.vstack((rul_data, ruls))

    return data_data, rul_data
#     return data_data, (rul_data - 75) / 75

# create dataset wrapper
def create_datasets(df, ws):
    sensor_data, rul_data = create_dataset_for_experiment(df, ws)

    padded_sensor_data = sensor_data.copy() 

    # Calculate X(t) 
    X_t = padded_sensor_data.copy()

    # Calculate y(t) 
    y_t = rul_data.copy()

    return X_t, y_t 



In [ ]:
class DataScaler(object):
    
    def get_scaler(self, scaler_name=None):
        ret = None
        if scaler_name == 'standard':
            ret = StandardScaler()
        elif scaler_name == 'minmax':
            ret = MinMaxScaler()
        elif scaler_name == 'minmax01':
            ret = MinMaxScaler()
        elif scaler_name == 'minmax-11':
            ret = MinMaxScaler(feature_range=(-1,1))
        elif scaler_name == 'robust':
            ret = RobustScaler()
        return ret

    def __init__(self, scaler_name):
        self.profiles = []
        self.scalers = []
        self.scaler_name = scaler_name
    
    def fit(self, X_df, y=0):
        self.scalers = {}
        self.profiles = X_df['Kmeans_Profile'].unique()
        # Full dataset fit
        for profile in self.profiles:
            sensors_readings = X_df[(X_df['Kmeans_Profile']==profile)].filter(sensor_list)          # Get sensor readings
            state_scaler = self.get_scaler(self.scaler_name).fit(sensors_readings)              # Fit scaler
            self.scalers[profile] = state_scaler                                                   # Add to sclaer_list for further reference
        
        return self
     
    def transform(self, X_df, y=0):
        # Full dataset transform
        for profile in self.profiles:
            sensors_readings = X_df[(X_df['Kmeans_Profile']==profile)].filter(sensor_list)          # Get sensor readings
            if (sensors_readings.shape[0] == 0): continue # No matching profiles found in X_df
            cols = sensors_readings.columns
            normalized_sensor_readings = self.scalers[profile].transform(sensors_readings)      # transform sensor readings
            X_df.loc[(X_df['Kmeans_Profile']==profile), cols] = normalized_sensor_readings                  # record transformed values

        return X_df


# RUL Assignment <a class="anchor" id="rul-assignment"></a>

For this challenge we construct Linear, Piecewise Linear (PwL), and rounded PwL. We will use the one that generates minimum penalty score.

In [ ]:
#
# Create extra features for each data file, and generate a DataFrame.
#
def convert_to_df(experimentId, path=None, size=1, ratio=0.4, profile=0):
# Initial columns for the data.
    cols = ['Time', 'Flow_Rate', "Upstream_Pressure", "Downstream_Pressure"]
    df = pd.DataFrame()
    
    fname = path + "Sample" + str(experimentId).zfill(2) + ".csv"
    if verbose: print ("Reading:", fname)
        
    temp_df = pd.read_csv(fname)
    temp_df.columns = cols
# count is the number of readings for the experiment
    count = temp_df['Time'].count()
    temp_df['ExperimentID'] = experimentId
    temp_df['ReadingID'] = list(range(1, count+1))
    temp_df['Particle_Size'] = size
    temp_df['Ratio'] = ratio
    temp_df['Profile'] = profile
    temp_df['Pressure_Drop'] = temp_df.Upstream_Pressure - temp_df.Downstream_Pressure

# Find the clogged index
    index = np.argmax(running_mean(temp_df.Pressure_Drop.values, 5) > 20)

# Linear RUL Assignment
    temp_df['Linear'] = None # in seconds
    pos_rul_list = [c for c in range(index+1)]
    pos_rul_list.reverse()
    temp_df.iloc[:index+1, temp_df.columns.get_loc('Linear')] = pos_rul_list
    neg_rul_list = [-c for c in range(count - index)]
    temp_df.iloc[index:, temp_df.columns.get_loc('Linear')] = neg_rul_list
    temp_df['Linear'] = temp_df['Linear'] / 10.0 # Sampled at 10 Hz

# Piecewise Linear (PwL) RUL Assignment
    temp_df['PwL'] = temp_df['Linear'].copy()
    temp_df.loc[temp_df['PwL'] > GLOBAL_INIT_RUL, 'PwL'] = GLOBAL_INIT_RUL

# PwL through fault
    temp_df['PwL_Fault'] = temp_df['Linear'].copy()
    f_index = fault_estimation(temp_df)
    max_rul_for_exp = temp_df.iloc[f_index].Linear
    
    # fit a line b/w (0, GLOBAL_INIT_RUL) and (f_index, max_rul_for_exp)
#     print ([0, f_index], [GLOBAL_INIT_RUL, max_rul_for_exp])
    z0 = np.polyfit([0, f_index], [GLOBAL_INIT_RUL, max_rul_for_exp], 1) 

    line_func = np.poly1d(z0) # create function flow line params
    x_line = line_func(list(range(f_index-1))).round(1) #.astype(np.int)

    temp_df.iloc[:f_index-1, temp_df.columns.get_loc("PwL_Fault")] = x_line

    df = pd.concat([df, temp_df])
    return df

In [ ]:
# Convert dataset into a DataFrame
def load_dataset(dataset_list):
    print ("Loading:", dataset_list)

    dataset_df = pd.DataFrame()
    for e in dataset_list:
        df = convert_to_df(e, **dataset_params[e])
        dataset_df = pd.concat([dataset_df, df], ignore_index=True)

    return dataset_df

# Profile Assignment

In [ ]:
def get_kmeans_model(train_df):
    sample_df = train_df.sample(frac=0.05)
    X_train_for_kmeans = sample_df[experiment_params].values
    kmeans_model = KMeans(n_clusters=n_profiles, max_iter=10)
    kmeans_model.fit_predict(X_train_for_kmeans)

    return kmeans_model

## Training <a class="anchor" id="ml-training"></a>

In [ ]:
def train_model_fn(train_df, scale_type, window_size, final_model_path):
    print ("Final training started...")

    X_train_df = train_df.copy()

    kmeans_model = get_kmeans_model(X_train_df)

    X_train_df['Kmeans_Profile'] = kmeans_model.predict(X_train_df[experiment_params].values)
    
    preprocessing_pipe = Pipeline([('scaler', DataScaler(scale_type))])

    # Transform training and validation datasets        
    X_train_df = preprocessing_pipe.fit(X_train_df).transform(X_train_df)

    X_train_t, y_train= create_datasets(X_train_df, window_size)
    X_train_t = X_train_t.astype(np.float32) 

    if verbose: 
        print (X_train_t.shape, y_train.shape)        
        print("---------------------------")

    # Model initialization        

    model = CatBoostRegressor(iterations=1000, thread_count=8)    
    model.fit(X=X_train_t, y=y_train, verbose=False) 

    pickle.dump((kmeans_model, preprocessing_pipe, model), open(final_model_path, "wb"))

    print('Training done.')

## Evaluation <a class="anchor" id="ml-evaluation"></a> 

In [ ]:
def evaluate_model_fn(validation_df, scale_type, window_size, final_model_path):
    print ("Final evaluation started...")

    with open(final_model_path, "rb") as f:
        kmeans_model, preprocessing_pipe, model = pickle.load(f)

    X_val_df = validation_df.copy()

    X_val_df['Kmeans_Profile'] = kmeans_model.predict(X_val_df[experiment_params].values)
    
    # This should be the proper way of scaling data:
    X_val_df = preprocessing_pipe.transform(X_val_df)


    X_val_t, y_val = create_datasets(X_val_df, window_size)
    X_val_t = X_val_t.astype(np.float32) 

    if verbose: 
        print (X_val_t.shape, y_val.shape)        
        print("---------------------------")

    test_prediction = model.predict(X_val_t) 

    result = mean_absolute_error((y_val)[::100], test_prediction[::100])
    print ('Evaluation done.')
    
    return result

In [ ]:
results_df = pd.DataFrame()

In [ ]:
# RUL assignment methods
ram_list1 = ['Linear'] # Neutral from RUL_Max
ram_list2 = ['PwL', 'PwL_Fault']

# Other hiperparameters
dp_list = [25, 50, 75, 100] # Data percentage
rm_list = [100, 125, 150] # RUL Max
ws_list = [5, 10, 15, 20, 25, 30, 40, 50] # Window size

total_runs = len(ram_list1) * len(dp_list) * len(ws_list) + \
                len(ram_list2) * len(dp_list) * len(rm_list) * len(ws_list)

run_counter = 0

In [ ]:
@ex.main
def ex_main(_run, RUL_Max, scale_type, RUL_Assignment_Method, window_size, train_model, eval_model,
         data_percent, training_list, validating_list, final_model_path):

    global RUL_Assignment
    global GLOBAL_INIT_RUL
    global results_df
    global total_runs
    global run_counter
    
    print ('===========================================================================')
    print ('Run:', run_counter+1, '/', total_runs)
    print ('RAM:', RUL_Assignment_Method, 'DP:', data_percent, 'RM:', RUL_Max, 'WS:', window_size)
    run_counter += 1
    
#     print_config(_run)
    RUL_Assignment = RUL_Assignment_Method
    GLOBAL_INIT_RUL = RUL_Max
    
    
    if (train_model):
        train_df = load_dataset(training_list)
        train_model_fn(train_df, scale_type, window_size, final_model_path)
    
    if (eval_model):
        validation_df = load_dataset(validating_list)
        score = evaluate_model_fn(validation_df, scale_type, window_size, final_model_path)
        
        result = {
            'ML_Algorithm': 'CatBoost',
            'RUL_Max': RUL_Max,
            'scale_type': scale_type,
            'RUL_Assignment_Method': RUL_Assignment_Method,
            'window_size': window_size,
            'data_percent': data_percent,
            'score': score
        }
        print (score)
        results_df = results_df.append(result, ignore_index=True)


In [ ]:
%%time
# Takes ~1 hr. on Intel® Xeon® Processor E5-1660 @ 3.2 GHz

# Linear RUL Assignment
for rm in [150]:
    for ram in ram_list1:
        for dp in dp_list:
            for ws in ws_list:
                ex.run(config_updates={'RUL_Assignment_Method':ram,
                       'data_percent': dp,
                       'RUL_Max': rm, 
                       'window_size': ws,
                      })
                results_df.to_excel("CB_results_kmeans_profile+.xlsx")

# PwL & PwL_Fault RUL Assignment                    
for rm in rm_list:
    for ram in ram_list2:
        for dp in dp_list:
            for ws in ws_list:
                ex.run(config_updates={'RUL_Assignment_Method':ram,
                       'data_percent': dp,
                       'RUL_Max': rm, 
                       'window_size': ws,
                      })
                results_df.to_excel("CB_results_kmeans_profile+.xlsx")
